#  NLCテストツール

## 環境固有情報の設定
以下の3つのセルは環境に応じて書き換えて利用します。

* 変数定義
* COS資格情報
* NLC資格上場

In [ ]:
#変数定義
#

# 入力EXCELファイル名
infile = 'nlc-test-sample.xlsx'

# 出力EXCELファイル名
outfile = 'nlc-test-sample-output.xlsx'

# 分類クラス数
# 出力の分類クラス、確信度を1件あたり何クラス分取得するかの値
n = 3

# model id
# 各自のトレーニング済みクラスのIDを指定します。
model_id = 'xxxxx'


In [ ]:
# COS資格情報
# 
# 各自のCOS資格情報に置き換えます。
# 資格情報のコピペでいいですが、最後の'File'のところはinfileに置き換えて下さい。

cos_credentials = {
    'IAM_SERVICE_ID': 'XXXX',
    'IBM_API_KEY_ID': 'XXXX',
    'ENDPOINT': 'XXXX',
    'IBM_AUTH_ENDPOINT': 'https://iam.ng.bluemix.net/oidc/token',
    'BUCKET': 'XXXX',
    'FILE': infile
}


In [ ]:
# NLC資格情報
# NLC資格情報をコピペで置き換えます。

nlc_credentials = {
  "apikey": "XXXX",
  "iam_apikey_description": "XXXX",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "XXXX",
  "iam_serviceid_crn": "XXXX",
  "url": "https://gateway.watsonplatform.net/natural-language-classifier/api"
}

## COSからローカルにファイルコピー

In [ ]:
# COSオブジェクトの生成
# 資格情報が正しければ、cosオブジェクトが生成されます。

from ibm_botocore.client import Config
import ibm_boto3

cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=cos_credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=cos_credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=cos_credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=cos_credentials['ENDPOINT'])

In [ ]:
# EXCELファイルをNotebook 仮想OSの$HOMEにコピー

filename = cos_credentials['FILE']
cos.download_file(Bucket=cos_credentials['BUCKET'],Key=filename,Filename=filename)

 ## Excelをデータフレームに読み込み

In [ ]:
# !pip install xlrd | tail -n 1
# 現在のStudioでは導入済みなので不要

In [ ]:
import pandas as pd
import json
import numpy as np
from IPython.core.display import display

In [ ]:
# Excel読込み

df1 = pd.read_excel(infile)
display(df1.head())

## NLC 呼出し準備

In [ ]:
!pip install ibm-watson | tail -n 1

In [ ]:
# NLCインスタンス生成
# 資格情報が正しければnlcインスタンスが生成されます。

from ibm_watson import NaturalLanguageClassifierV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

apikey = nlc_credentials['apikey']
url = nlc_credentials['url']
authenticator = IAMAuthenticator(apikey)
nlc = NaturalLanguageClassifierV1(
    authenticator=authenticator
)
nlc.set_service_url(url)

### 呼出しテスト

In [ ]:
# 入力Excelのtext列最初のテキストを抽出
text = df1['text'][0]

# 結果確認
print('【入力テキスト】')
print(text)
print('')

# このテキストを使ってNLC呼出し
ret = nlc.classify(model_id, text).get_result()

# 結果確認
print('【出力JSON】')
print(json.dumps(ret, indent=2,ensure_ascii=False))

## 全テストデータに対してAPI呼出し

In [ ]:
# columns生成
columns = []
for i in range(n):
    item = 'item%d' % i
    conf = 'conf%d' % i
    columns = columns + [item, conf]

# values生成
vv = []

for i in range(len(df1)):
    text = df1['text'][i]
    ret = nlc.classify(model_id, text).get_result()

    ret_values = ret['classes']
    values = []
    for i in range(n):
        item = ret_values[i]
        cls = item['class_name']
        val = item['confidence']
        values = values + [cls, val]
    
    vv = vv + [values]

# DataFrame生成
df_out = pd.DataFrame(vv, columns = columns)
display(df_out)

In [ ]:
# 元データと連結

df_total = pd.concat([df1, df_out], axis=1)
display(df_total)

## Excelに書き出し

In [ ]:
# Excelに書き出し
df_total.to_excel(outfile, index=False)

# COSに書き戻し
cos.upload_file(Filename=outfile,Bucket=cos_credentials['BUCKET'],Key=outfile)

## Confusion Matix(混同行列)の表示

In [ ]:
# 正解データ
classes_t = df_total['class'].values

# 予測データ
classes_p = df_total['item0'].values

print(classes_t)
print(classes_p)

In [ ]:
# ラベルリストの作成

labels = list(set(classes_t))
print(labels)

In [ ]:
# 混同行列の生成
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(classes_t, classes_p, labels=labels)

# データフレームに変換
cm_labeled = pd.DataFrame(cm, columns=labels, index=labels)
display(cm_labeled)